In [1]:
%load_ext watermark
%watermark

Last updated: 2024-06-04T23:06:48.238469-05:00

Python implementation: CPython
Python version       : 3.10.11
IPython version      : 8.13.2

Compiler    : MSC v.1916 64 bit (AMD64)
OS          : Windows
Release     : 10
Machine     : AMD64
Processor   : Intel64 Family 6 Model 158 Stepping 10, GenuineIntel
CPU cores   : 8
Architecture: 64bit



In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing, feature_extraction
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline, FeatureUnion
#import category_encoders
from sklearn.model_selection import cross_validate
from category_encoders import OneHotEncoder
from category_encoders.ordinal import OrdinalEncoder
from sklearn.pipeline import FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.compose import ColumnTransformer

In [3]:
consumidores = pd.read_csv("C:/Users/Danny/Desktop/Analisis tesis/4 - Pipelines/data/df_consumo_varObjetivo.csv")

In [4]:
consumidores.head()

,departamento,residentes_hogar,d2_04_num_hijos,per_edad_tipo,estrato_tipo,situacion_tipo,frecuencia_consumo_marihuana_tipo,frecuencia_consumo_cocaina_tipo,frecuencia_consumo_basuco_tipo,per_sexo_tipo,...,g_06_a_posibilidad_conseguir_marihuana_tipo,g_06_b_posibilidad_conseguir_cocaina_tipo,g_06_c_posibilidad_conseguir_basuco_tipo,g_07_alguien_ofrecio_comprar_probar_sp_tipo,g_01_a_num_familiares_consumen_sp_imp_tipo,g_02_a_num_amigos_consumen_sp_imp_tipo,g_08_a_ofrecieron_marihuana_imp_tipo,g_08_b_ofrecieron_cocaina_imp_tipo,g_08_c_ofrecieron_basuco_imp_tipo,CatRiesgo
0,Guaviare,5,0,Joven,Estrato bajo,Trabajando,Nunca,Nunca,Nunca,Hombre,...,g_06_a_No_conoce_facilidad_dificultad,g_06_b_No_conoce_facilidad_dificultad,g_06_c_No_conoce_facilidad_dificultad,g_07_No,g_01_a_No_aplica,g_02_a_Dos_o_mas,g_08_a_No_aplica,g_08_b_No_aplica,g_08_c_No_aplica,0
1,Cordoba,4,3,Adulto,Estrato bajo,Trabajando,Nunca,Nunca,Nunca,Mujer,...,g_06_a_Seria_facil,g_06_b_Seria_facil,g_06_c_Seria_facil,g_07_No,g_01_a_No_aplica,g_02_a_No_aplica,g_08_a_No_aplica,g_08_b_No_aplica,g_08_c_No_aplica,0
2,Caqueta,4,2,Adulto,Estrato bajo,Trabajando,Nunca,Nunca,Nunca,Hombre,...,g_06_a_No_conoce_facilidad_dificultad,g_06_b_No_conoce_facilidad_dificultad,g_06_c_No_conoce_facilidad_dificultad,g_07_No,g_01_a_No_aplica,g_02_a_No_aplica,g_08_a_No_aplica,g_08_b_No_aplica,g_08_c_No_aplica,0
3,Bolivar,6,0,Adulto,Estrato alto,Trabajando,Nunca,Nunca,Nunca,Mujer,...,g_06_a_No_podria_conseguir,g_06_b_No_podria_conseguir,g_06_c_No_podria_conseguir,g_07_No,g_01_a_No_aplica,g_02_a_No_aplica,g_08_a_No_aplica,g_08_b_No_aplica,g_08_c_No_aplica,0
4,Tolima,4,1,Joven,Estrato bajo,Buscando trabajo,Nunca,Nunca,Nunca,Mujer,...,g_06_a_Seria_facil,g_06_b_Seria_facil,g_06_c_Seria_facil,g_07_No,g_01_a_No_aplica,g_02_a_Dos_o_mas,g_08_a_No_aplica,g_08_b_No_aplica,g_08_c_No_aplica,1


En este modelo vamos a modificar cada variable en función de su tipo. Al conjunto de pasos que siguen los datos se le llama comúnmente **Pipelines** (literalmente, sistemas de tuberias).

In [5]:
consumidores.shape

(10000, 40)

**INCISO**

vamos a modificar dos transformadores de scikitlearn para que sean compatibles con pipelines. Este paso es necesario en la version actual de scikit-learn

In [6]:
variable_objetivo = ["CatRiesgo"]
variables_independientes = consumidores.drop(variable_objetivo, axis=1).columns

In [7]:
datos_numericos = consumidores[variables_independientes].select_dtypes([int, float])
col_numericas = datos_numericos.columns
col_no_numericas = consumidores[variables_independientes].select_dtypes([object]).columns

dict_var_ordinales = {
    "per_edad_tipo": ["Adolescente", "Joven", "Adulto", "Persona mayor"],
    "estrato_tipo":["Estrato bajo", "Estrato medio", "Estrato alto"],
    "d_08_estado_salud_tipo":["d_08_Muy_malo","d_08_Malo","d_08_Regular","d_08_Bueno","d_08_Muy_bueno"],
    "frecuencia_consumo_marihuana_tipo":["Nunca", "Alguna vez en su vida", "Una vez en el año", "Algunas veces durante los últimos 12 meses", "Algunas veces mensualmente", "Algunas veces semanalmente", "Diariamente"],
    "frecuencia_consumo_cocaina_tipo":["Nunca", "Alguna vez en su vida", "Una vez en el año", "Algunas veces durante los últimos 12 meses", "Algunas veces mensualmente", "Algunas veces semanalmente", "Diariamente"],
    "frecuencia_consumo_basuco_tipo":["Nunca", "Alguna vez en su vida", "Una vez en el año", "Algunas veces durante los últimos 12 meses", "Algunas veces mensualmente", "Algunas veces semanalmente", "Diariamente"],
}

col_ordinales = list(dict_var_ordinales.keys())
datos_ordinales = consumidores[col_ordinales]
col_categoricas = list(set(col_no_numericas) - set(col_ordinales))
datos_categoricos = consumidores[col_categoricas]

Un pipeline de sklearn se define como una secuencia de pasos. Cada paso se define con una tupla de forma `[nombre del paso, transformador]`

Por ejemplo, si queremos crear un pipeline que procese las variables numéricas, primero imputándolas y después estandarizandolas, podriamos crear un pipeline

Creamos ahora los pipelines para cada tipo de variable. Es necesario aclarar que se va ha hacer uso de la libreria Column Transformer par ael procesamiento de variables categoricas y categoricas ordinales

In [8]:
transformador_numerico = Pipeline([
    ("imputador_numerico", SimpleImputer(missing_values=np.nan, strategy='mean')),
    ("escalador_numerico", preprocessing.MinMaxScaler())
])

pipeline_numerico = ColumnTransformer([
    ("transformador_numerico", transformador_numerico, col_numericas),
])

pipeline_numerico.fit_transform(consumidores)[:5]

array([[0.30769231, 0.        ],
       [0.23076923, 0.25      ],
       [0.23076923, 0.16666667],
       [0.38461538, 0.        ],
       [0.23076923, 0.08333333]])

In [9]:
pipeline_numerico

ColumnTransformer(transformers=[('transformador_numerico',
                                 Pipeline(steps=[('imputador_numerico',
                                                  SimpleImputer()),
                                                 ('escalador_numerico',
                                                  MinMaxScaler())]),
                                 Index(['residentes_hogar', 'd2_04_num_hijos'], dtype='object'))])

Ahora se procede a crear el pipeline categorico

In [10]:
transformador_categorico = ColumnTransformer([
    ("transformador_categorico", OneHotEncoder(handle_unknown="impute"), col_categoricas)
])

pipeline_categorico = Pipeline([
    ("transformador_categorico", transformador_categorico)
])

pipeline_categorico.fit_transform(consumidores)[:5]

array([[1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0,
        0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0,
        0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0,
        0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1,
        0, 0, 0, 1, 0],
       [1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0,
        0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0,
        0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0,
        0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0,
        1, 0, 0, 1, 0],
       [1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,

In [11]:
pipeline_categorico

Pipeline(steps=[('transformador_categorico',
                 ColumnTransformer(transformers=[('transformador_categorico',
                                                  OneHotEncoder(handle_unknown='impute'),
                                                  ['d2_01_etnia_tipo',
                                                   'g_03_curiosidad_probar_sp_tipo',
                                                   'g_04_probaria_sp_tipo',
                                                   'd2_03_estado_civil_tipo',
                                                   'd_11_h_conocimiento_riesgo_fumar_marihuana_frecuentemente_tipo',
                                                   'vive_padre_hogar_tipo',
                                                   'g_02_a_num_ami...
                                                   'd_11_n_conocimiento_riesgo_fumar_basuco_frecuentemente_tipo',
                                                   'g_08_b_ofrecieron_cocaina_imp_tipo',
                                                   'g_06_a_posibilidad_conseguir_marihuana_tipo',
                                                   'g_01_familiares_consumen_sp_tipo',
                                                   'situacion_tipo',
                                                   'd_09_deprimido_tipo',
                                                   'departamento',
                                                   'g_02_amigos_consumen_sp_tipo',
                                                   'g_07_alguien_ofrecio_comprar_probar_sp_tipo',
                                                   'vivienda_tipo',
                                                   'g_06_c_posibilidad_conseguir_basuco_tipo', ...])]))])

`columnTransformer` requiere un mapeado específico para cada columna, indicando el número por el que queremos reemplazar cada nivel de cada variable ordinal. Si no especificamos un mapa, asume que el orden es aleatorio.

In [12]:
niveles_ordinales = []
for col, levels in dict_var_ordinales.items():
    niveles_ordinales.append({"col": col, 
                              "mapping": dict(zip(levels, range(len(levels))))})

In [13]:
transformador_ordinal = ColumnTransformer([
    ("transformador_ordinal", OrdinalEncoder(mapping=niveles_ordinales), col_ordinales)
])

pipeline_ordinal = Pipeline([
    ("transformador_ordinal", transformador_ordinal)
])
pipeline_ordinal.fit_transform(consumidores)[:5]

array([[1, 0, 1, 0, 0, 0],
       [2, 0, 2, 0, 0, 0],
       [2, 0, 3, 0, 0, 0],
       [2, 2, 3, 0, 0, 0],
       [1, 0, 3, 0, 0, 0]])

Ya tenemos una manera de, dado un conjunto de datos, separarlos y aplicar distintas transformaciones a cada variable. Nos falta una manera de, una vez se han transformado, reunirlas de nuevo.

Para ello podemos usar `FeatureUnion`, que simplemente toma un conjunto de pasos de un pipeline y los une.

In [14]:
pipeline_procesado = FeatureUnion([
    ('variables_numericas', pipeline_numerico),
    ('variables_ordinales', pipeline_ordinal),
    ('variables_categoricas', pipeline_categorico),
])

In [15]:
pipeline_procesado

FeatureUnion(transformer_list=[('variables_numericas',
                                ColumnTransformer(transformers=[('transformador_numerico',
                                                                 Pipeline(steps=[('imputador_numerico',
                                                                                  SimpleImputer()),
                                                                                 ('escalador_numerico',
                                                                                  MinMaxScaler())]),
                                                                 Index(['residentes_hogar', 'd2_04_num_hijos'], dtype='object'))])),
                               ('variables_ordinales',
                                Pipeline(steps=[('transformador_ordinal',
                                                 ColumnTransfor...
                                                                                   'd_11_n_conocimiento_riesgo_fumar_basuco_frecuentemente_tipo',
                                                                                   'g_08_b_ofrecieron_cocaina_imp_tipo',
                                                                                   'g_06_a_posibilidad_conseguir_marihuana_tipo',
                                                                                   'g_01_familiares_consumen_sp_tipo',
                                                                                   'situacion_tipo',
                                                                                   'd_09_deprimido_tipo',
                                                                                   'departamento',
                                                                                   'g_02_amigos_consumen_sp_tipo',
                                                                                   'g_07_alguien_ofrecio_comprar_probar_sp_tipo',
                                                                                   'vivienda_tipo',
                                                                                   'g_06_c_posibilidad_conseguir_basuco_tipo', ...])]))]))])

In [16]:
consumidores_procesado = pipeline_procesado.fit_transform(consumidores)
consumidores_procesado.shape

(10000, 145)

Finalmente, necesitamos añadir un estimador al final para predecir en base a los datos transformados, pero primero se separa los datos en datos de entrenamiento y datos de test

In [17]:
X_train, X_test, y_train, y_test = train_test_split(
     consumidores[variables_independientes], consumidores["CatRiesgo"],
     test_size=0.20, random_state=42
)

In [18]:
estimador = LogisticRegression(max_iter=1000)

Se debe realizar la separacion de datos en datos de entrenamiento y datos de test y ademas se debe establecer el estimador a usar en el modelo

In [19]:
pipeline_estimador = Pipeline([
    ('procesador', pipeline_procesado),
    ('estimador', estimador)
])

Para el fit se pasan los datos que se obtienen de entrenamiento y para el predict los datos de test

In [20]:
pipeline_estimador.fit(X_train, y_train)

Pipeline(steps=[('procesador',
                 FeatureUnion(transformer_list=[('variables_numericas',
                                                 ColumnTransformer(transformers=[('transformador_numerico',
                                                                                  Pipeline(steps=[('imputador_numerico',
                                                                                                   SimpleImputer()),
                                                                                                  ('escalador_numerico',
                                                                                                   MinMaxScaler())]),
                                                                                  Index(['residentes_hogar', 'd2_04_num_hijos'], dtype='object'))])),
                                                ('variables_ordinales',
                                                 Pipeline(steps=[('transfo...
                                                                                                    'g_08_b_ofrecieron_cocaina_imp_tipo',
                                                                                                    'g_06_a_posibilidad_conseguir_marihuana_tipo',
                                                                                                    'g_01_familiares_consumen_sp_tipo',
                                                                                                    'situacion_tipo',
                                                                                                    'd_09_deprimido_tipo',
                                                                                                    'departamento',
                                                                                                    'g_02_amigos_consumen_sp_tipo',
                                                                                                    'g_07_alguien_ofrecio_comprar_probar_sp_tipo',
                                                                                                    'vivienda_tipo',
                                                                                                    'g_06_c_posibilidad_conseguir_basuco_tipo', ...])]))]))])),
                ('estimador', LogisticRegression(max_iter=1000))])

In [21]:
predicciones = pipeline_estimador.predict(X_test)

In [22]:
predicciones

array([0, 2, 0, ..., 2, 0, 4], dtype=int64)

Se evalua el modelo segun lo obtenido

El beneficio de los pipelines, no solo es tener codigo mas legible y poder gestionar de forma ordenada todo el ciclo de vida del modelado, sino que los pipelines tienen todos los beneficios de los objetos de scikitlearn, por ejemplo, podemos usar validacion cruzada directamente con el pipeline.

In [23]:
resultados = {}

def exactitud_multietiqueta(estimador, X, y):
    preds = estimador.predict(X)
    return metrics.accuracy_score(y, preds)

def precision_multietiqueta(estimador, X, y):
    preds = estimador.predict(X)
    return metrics.precision_score(y, preds, average="weighted")

def exhaustividad_multietiqueta(estimador, X, y):
    preds = estimador.predict(X)
    return metrics.recall_score(y, preds, average="weighted")

def f1_multietiqueta(estimador, X, y):
    preds = estimador.predict(X)
    return metrics.f1_score(y, preds, average="weighted")

In [24]:
def evaluar_modelo(prmPipeline,X, y, semillas):
    restultados_estimador = {}
    restultados_estimador["exactitud"] = cross_validate(prmPipeline, X, y,
                                           scoring=exactitud_multietiqueta, n_jobs=-1, cv=semillas, return_train_score=True)
    
    restultados_estimador["precision"] = cross_validate(prmPipeline, X, y,
                                           scoring=precision_multietiqueta, n_jobs=-1, cv=semillas, return_train_score=True)
    
    restultados_estimador["exhaustividad"] = cross_validate(prmPipeline, X, y,
                                           scoring=exhaustividad_multietiqueta, n_jobs=-1, cv=semillas, return_train_score=True)
    
    restultados_estimador["f1_score"] = cross_validate(prmPipeline, X, y,
                                           scoring=f1_multietiqueta, n_jobs=-1, cv=semillas, return_train_score=True)
    return restultados_estimador

In [25]:
#Funcion para poder visuallizar los resultados de las metricas
def ver_resultados():
    df_res = pd.DataFrame()
    for modelo, metricas in resultados.items():
        for metrica, valores in metricas.items():
            for medida, arreglo in valores.items():
                # Calcula la media de cada array y agrega una nueva fila al DataFrame
                df_res.loc[f"{modelo}_{metrica}", medida] = np.mean(arreglo)
                
    return df_res

In [26]:
estimador_agregado = pipeline_estimador.named_steps['estimador']
resultados["rl"] = evaluar_modelo(estimador_agregado, consumidores_procesado, consumidores.CatRiesgo, 50)

In [27]:
ver_resultados()

,fit_time,score_time,test_score,train_score
rl_exactitud,14.741924,0.002741,0.980400,0.992369
rl_precision,14.511580,0.005860,0.981103,0.992360
rl_exhaustividad,14.600140,0.004940,0.980400,0.992369
rl_f1_score,14.440344,0.005501,0.980343,0.992360


### Optimizacion de hiperparametros

En esta sección se configuran los diferentes parámetros del modelo, con el objetivo de mejorar los resultados de las métricas.

In [28]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint
from scipy.stats import uniform, loguniform

In [29]:
param_dist_random = {
    'C': loguniform(0.01, 100),
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear', 'saga'],
    'max_iter': [1000, 2000, 3000, 5000]
}

In [30]:
busqueda_random = RandomizedSearchCV(estimator=estimador, 
                   param_distributions=param_dist_random,
                   scoring="accuracy", n_jobs=-1, n_iter=100)

In [31]:
pipeline_estimador_op = Pipeline([
    ('procesador', pipeline_procesado),
    ('estimador', busqueda_random)
])

In [32]:
pipeline_estimador_op.fit(X_train, y_train)

Pipeline(steps=[('procesador',
                 FeatureUnion(transformer_list=[('variables_numericas',
                                                 ColumnTransformer(transformers=[('transformador_numerico',
                                                                                  Pipeline(steps=[('imputador_numerico',
                                                                                                   SimpleImputer()),
                                                                                                  ('escalador_numerico',
                                                                                                   MinMaxScaler())]),
                                                                                  Index(['residentes_hogar', 'd2_04_num_hijos'], dtype='object'))])),
                                                ('variables_ordinales',
                                                 Pipeline(steps=[('transfo...
                                                                                                    'vive_madre_hogar_tipo', ...])]))]))])),
                ('estimador',
                 RandomizedSearchCV(estimator=LogisticRegression(max_iter=1000),
                                    n_iter=100, n_jobs=-1,
                                    param_distributions={'C': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x000001F27A168DC0>,
                                                         'max_iter': [1000,
                                                                      2000,
                                                                      3000,
                                                                      5000],
                                                         'penalty': ['l1',
                                                                     'l2'],
                                                         'solver': ['liblinear',
                                                                    'saga']},
                                    scoring='accuracy'))])

In [33]:
pipeline_estimador_op.predict(X_test)

array([0, 2, 0, ..., 2, 0, 4], dtype=int64)

In [34]:
print(busqueda_random.best_score_)
print(busqueda_random.best_estimator_)

0.9825000000000002
LogisticRegression(C=1.6762273626130806, max_iter=2000, penalty='l1',
                   solver='saga')


In [35]:
pipeline_estimador_op_best = Pipeline([
    ('procesador', pipeline_procesado),
    ('estimador', busqueda_random.best_estimator_)
])

In [36]:
pipeline_estimador_op_best.fit(X_train, y_train)

Pipeline(steps=[('procesador',
                 FeatureUnion(transformer_list=[('variables_numericas',
                                                 ColumnTransformer(transformers=[('transformador_numerico',
                                                                                  Pipeline(steps=[('imputador_numerico',
                                                                                                   SimpleImputer()),
                                                                                                  ('escalador_numerico',
                                                                                                   MinMaxScaler())]),
                                                                                  Index(['residentes_hogar', 'd2_04_num_hijos'], dtype='object'))])),
                                                ('variables_ordinales',
                                                 Pipeline(steps=[('transfo...
                                                                                                    'g_01_a_num_familiares_consumen_sp_imp_tipo',
                                                                                                    'per_sexo_tipo',
                                                                                                    'vivienda_tipo',
                                                                                                    'd2_03_estado_civil_tipo',
                                                                                                    'g_03_curiosidad_probar_sp_tipo',
                                                                                                    'd_12_c_presenta_problema_expendio_sp_barrio_tipo',
                                                                                                    'g_06_c_posibilidad_conseguir_basuco_tipo',
                                                                                                    'vive_padre_hogar_tipo',
                                                                                                    'vive_madre_hogar_tipo', ...])]))]))])),
                ('estimador',
                 LogisticRegression(C=1.6762273626130806, max_iter=2000,
                                    penalty='l1', solver='saga'))])

In [37]:
pipeline_estimador_op_best.predict(X_test)

array([0, 2, 0, ..., 2, 0, 4], dtype=int64)

In [38]:
estimador_best = pipeline_estimador_op_best.named_steps['estimador']
resultados["rl_best"] = evaluar_modelo(estimador_best, consumidores_procesado, consumidores.CatRiesgo, 50)
ver_resultados()

,fit_time,score_time,test_score,train_score
rl_exactitud,14.741924,0.002741,0.980400,0.992369
rl_precision,14.511580,0.005860,0.981103,0.992360
rl_exhaustividad,14.600140,0.004940,0.980400,0.992369
rl_f1_score,14.440344,0.005501,0.980343,0.992360
rl_best_exactitud,189.739768,0.001530,0.986000,0.997016
rl_best_precision,183.521629,0.004372,0.986594,0.997018
rl_best_exhaustividad,183.239200,0.004620,0.986000,0.997016
rl_best_f1_score,182.556387,0.004101,0.985960,0.997011


### Segunda optimizacion

In [39]:
param_dist_random2 = {
    'C': loguniform(0.01, 100),
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear', 'saga'],
    'max_iter': [1000, 2000, 3000, 5000],
    'dual': [False],
    'fit_intercept': [True, False]
}

In [40]:
busqueda_random2 = RandomizedSearchCV(estimator=estimador, 
                    param_distributions=param_dist_random2,
                   scoring="accuracy", n_jobs=-1, n_iter=100)

In [41]:
pipeline_estimador_op2 = Pipeline([
    ('procesador', pipeline_procesado),
    ('estimador', busqueda_random2)
])

In [42]:
pipeline_estimador_op2.fit(X_train, y_train)

Pipeline(steps=[('procesador',
                 FeatureUnion(transformer_list=[('variables_numericas',
                                                 ColumnTransformer(transformers=[('transformador_numerico',
                                                                                  Pipeline(steps=[('imputador_numerico',
                                                                                                   SimpleImputer()),
                                                                                                  ('escalador_numerico',
                                                                                                   MinMaxScaler())]),
                                                                                  Index(['residentes_hogar', 'd2_04_num_hijos'], dtype='object'))])),
                                                ('variables_ordinales',
                                                 Pipeline(steps=[('transfo...
                ('estimador',
                 RandomizedSearchCV(estimator=LogisticRegression(max_iter=1000),
                                    n_iter=100, n_jobs=-1,
                                    param_distributions={'C': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x000001F27A0B68C0>,
                                                         'dual': [False],
                                                         'fit_intercept': [True,
                                                                           False],
                                                         'max_iter': [1000,
                                                                      2000,
                                                                      3000,
                                                                      5000],
                                                         'penalty': ['l1',
                                                                     'l2'],
                                                         'solver': ['liblinear',
                                                                    'saga']},
                                    scoring='accuracy'))])

In [43]:
pipeline_estimador_op2.predict(X_test)

array([0, 2, 0, ..., 2, 0, 4], dtype=int64)

In [44]:
print(busqueda_random2.best_score_)
print(busqueda_random2.best_estimator_)

0.982375
LogisticRegression(C=1.2915081115686997, max_iter=5000, penalty='l1',
                   solver='saga')


In [45]:
pipeline_estimador_op_best2 = Pipeline([
    ('procesador', pipeline_procesado),
    ('estimador', busqueda_random2.best_estimator_)
])

In [46]:
pipeline_estimador_op_best2.fit(X_train, y_train)

Pipeline(steps=[('procesador',
                 FeatureUnion(transformer_list=[('variables_numericas',
                                                 ColumnTransformer(transformers=[('transformador_numerico',
                                                                                  Pipeline(steps=[('imputador_numerico',
                                                                                                   SimpleImputer()),
                                                                                                  ('escalador_numerico',
                                                                                                   MinMaxScaler())]),
                                                                                  Index(['residentes_hogar', 'd2_04_num_hijos'], dtype='object'))])),
                                                ('variables_ordinales',
                                                 Pipeline(steps=[('transfo...
                                                                                                    'g_01_a_num_familiares_consumen_sp_imp_tipo',
                                                                                                    'per_sexo_tipo',
                                                                                                    'vivienda_tipo',
                                                                                                    'd2_03_estado_civil_tipo',
                                                                                                    'g_03_curiosidad_probar_sp_tipo',
                                                                                                    'd_12_c_presenta_problema_expendio_sp_barrio_tipo',
                                                                                                    'g_06_c_posibilidad_conseguir_basuco_tipo',
                                                                                                    'vive_padre_hogar_tipo',
                                                                                                    'vive_madre_hogar_tipo', ...])]))]))])),
                ('estimador',
                 LogisticRegression(C=1.2915081115686997, max_iter=5000,
                                    penalty='l1', solver='saga'))])

In [47]:
pipeline_estimador_op_best2.predict(X_test)

array([0, 2, 0, ..., 2, 0, 4], dtype=int64)

In [48]:
estimador_best2 = pipeline_estimador_op_best2.named_steps['estimador']
resultados["rl_best_2"] = evaluar_modelo(estimador_best2, consumidores_procesado, consumidores.CatRiesgo, 50)
ver_resultados()

,fit_time,score_time,test_score,train_score
rl_exactitud,14.741924,0.002741,0.980400,0.992369
rl_precision,14.511580,0.005860,0.981103,0.992360
rl_exhaustividad,14.600140,0.004940,0.980400,0.992369
rl_f1_score,14.440344,0.005501,0.980343,0.992360
rl_best_exactitud,189.739768,0.001530,0.986000,0.997016
rl_best_precision,183.521629,0.004372,0.986594,0.997018
rl_best_exhaustividad,183.239200,0.004620,0.986000,0.997016
rl_best_f1_score,182.556387,0.004101,0.985960,0.997011
rl_best_2_exactitud,217.689699,0.001902,0.985300,0.996088
rl_best_2_precision,218.118712,0.003961,0.985912,0.996088


### Tercera optimizacion

In [49]:
param_dist_random3 = {
    'C': loguniform(0.01, 100),
    'penalty': ['l1'],
    'solver': ['saga'],
    'max_iter': [1000, 2000, 5000, 7000],
    'dual': [False],
    'fit_intercept': [True, False],
    'class_weight': [None, 'balanced'],
    'tol': [1e-4, 1e-3, 1e-2],
}

In [50]:
busqueda_random3 = RandomizedSearchCV(estimator=estimador, 
                    param_distributions=param_dist_random3,
                   scoring="accuracy", n_jobs=-1, n_iter=100)

In [51]:
pipeline_estimador_op3 = Pipeline([
    ('procesador', pipeline_procesado),
    ('estimador', busqueda_random3)
])

In [52]:
pipeline_estimador_op3.fit(X_train, y_train)

Pipeline(steps=[('procesador',
                 FeatureUnion(transformer_list=[('variables_numericas',
                                                 ColumnTransformer(transformers=[('transformador_numerico',
                                                                                  Pipeline(steps=[('imputador_numerico',
                                                                                                   SimpleImputer()),
                                                                                                  ('escalador_numerico',
                                                                                                   MinMaxScaler())]),
                                                                                  Index(['residentes_hogar', 'd2_04_num_hijos'], dtype='object'))])),
                                                ('variables_ordinales',
                                                 Pipeline(steps=[('transfo...
                 RandomizedSearchCV(estimator=LogisticRegression(max_iter=1000),
                                    n_iter=100, n_jobs=-1,
                                    param_distributions={'C': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x000001F27AF16AD0>,
                                                         'class_weight': [None,
                                                                          'balanced'],
                                                         'dual': [False],
                                                         'fit_intercept': [True,
                                                                           False],
                                                         'max_iter': [1000,
                                                                      2000,
                                                                      5000,
                                                                      7000],
                                                         'penalty': ['l1'],
                                                         'solver': ['saga'],
                                                         'tol': [0.0001, 0.001,
                                                                 0.01]},
                                    scoring='accuracy'))])

In [53]:
pipeline_estimador_op3.predict(X_test)

array([0, 2, 0, ..., 2, 0, 4], dtype=int64)

In [54]:
print(busqueda_random3.best_score_)
print(busqueda_random3.best_estimator_)

0.9817499999999999
LogisticRegression(C=5.458055980919349, fit_intercept=False, max_iter=5000,
                   penalty='l1', solver='saga')


In [55]:
pipeline_estimador_op_best3 = Pipeline([
    ('procesador', pipeline_procesado),
    ('estimador', busqueda_random3.best_estimator_)
])

In [56]:
pipeline_estimador_op_best3.fit(X_train, y_train)

Pipeline(steps=[('procesador',
                 FeatureUnion(transformer_list=[('variables_numericas',
                                                 ColumnTransformer(transformers=[('transformador_numerico',
                                                                                  Pipeline(steps=[('imputador_numerico',
                                                                                                   SimpleImputer()),
                                                                                                  ('escalador_numerico',
                                                                                                   MinMaxScaler())]),
                                                                                  Index(['residentes_hogar', 'd2_04_num_hijos'], dtype='object'))])),
                                                ('variables_ordinales',
                                                 Pipeline(steps=[('transfo...
                                                                                                    'per_sexo_tipo',
                                                                                                    'vivienda_tipo',
                                                                                                    'd2_03_estado_civil_tipo',
                                                                                                    'g_03_curiosidad_probar_sp_tipo',
                                                                                                    'd_12_c_presenta_problema_expendio_sp_barrio_tipo',
                                                                                                    'g_06_c_posibilidad_conseguir_basuco_tipo',
                                                                                                    'vive_padre_hogar_tipo',
                                                                                                    'vive_madre_hogar_tipo', ...])]))]))])),
                ('estimador',
                 LogisticRegression(C=5.458055980919349, fit_intercept=False,
                                    max_iter=5000, penalty='l1',
                                    solver='saga'))])

In [57]:
pipeline_estimador_op_best3.predict(X_test)

array([0, 2, 0, ..., 2, 0, 4], dtype=int64)

In [58]:
estimador_best3 = pipeline_estimador_op_best3.named_steps['estimador']
resultados["rl_best_3"] = evaluar_modelo(estimador_best3, consumidores_procesado, consumidores.CatRiesgo, 50)
ver_resultados()

,fit_time,score_time,test_score,train_score
rl_exactitud,14.741924,0.002741,0.980400,0.992369
rl_precision,14.511580,0.005860,0.981103,0.992360
rl_exhaustividad,14.600140,0.004940,0.980400,0.992369
rl_f1_score,14.440344,0.005501,0.980343,0.992360
rl_best_exactitud,189.739768,0.001530,0.986000,0.997016
rl_best_precision,183.521629,0.004372,0.986594,0.997018
rl_best_exhaustividad,183.239200,0.004620,0.986000,0.997016
rl_best_f1_score,182.556387,0.004101,0.985960,0.997011
rl_best_2_exactitud,217.689699,0.001902,0.985300,0.996088
rl_best_2_precision,218.118712,0.003961,0.985912,0.996088


### Cuarta optimizacion

In [59]:
param_dist_random4 = {
    'C': loguniform(0.01, 100),
    'penalty': ['l1'],
    'solver': ['saga'],
    'max_iter': [1000, 2000, 5000, 7000, 9000, 11000, 13000],
    'dual': [False],
    'fit_intercept': [True, False],
    'class_weight': [None, 'balanced'],
    'tol': [1e-4, 1e-3, 1e-2]
}

In [60]:
busqueda_random4 = RandomizedSearchCV(estimator=estimador, 
                    param_distributions=param_dist_random4,
                   scoring="accuracy", n_jobs=-1, n_iter=100)

In [61]:
pipeline_estimador_op4 = Pipeline([
    ('procesador', pipeline_procesado),
    ('estimador', busqueda_random4)
])

In [62]:
pipeline_estimador_op4.fit(X_train, y_train)

Pipeline(steps=[('procesador',
                 FeatureUnion(transformer_list=[('variables_numericas',
                                                 ColumnTransformer(transformers=[('transformador_numerico',
                                                                                  Pipeline(steps=[('imputador_numerico',
                                                                                                   SimpleImputer()),
                                                                                                  ('escalador_numerico',
                                                                                                   MinMaxScaler())]),
                                                                                  Index(['residentes_hogar', 'd2_04_num_hijos'], dtype='object'))])),
                                                ('variables_ordinales',
                                                 Pipeline(steps=[('transfo...
                 RandomizedSearchCV(estimator=LogisticRegression(max_iter=1000),
                                    n_iter=100, n_jobs=-1,
                                    param_distributions={'C': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x000001F27A0B5BD0>,
                                                         'class_weight': [None,
                                                                          'balanced'],
                                                         'dual': [False],
                                                         'fit_intercept': [True,
                                                                           False],
                                                         'max_iter': [1000,
                                                                      2000,
                                                                      5000,
                                                                      7000,
                                                                      9000,
                                                                      11000,
                                                                      13000],
                                                         'penalty': ['l1'],
                                                         'solver': ['saga'],
                                                         'tol': [0.0001, 0.001,
                                                                 0.01]},
                                    scoring='accuracy'))])

In [63]:
pipeline_estimador_op4.predict(X_test)

array([0, 2, 0, ..., 2, 0, 4], dtype=int64)

In [64]:
print(busqueda_random4.best_score_)
print(busqueda_random4.best_estimator_)

0.9831249999999999
LogisticRegression(C=2.3118996566114403, max_iter=13000, penalty='l1',
                   solver='saga')


In [65]:
pipeline_estimador_op_best4 = Pipeline([
    ('procesador', pipeline_procesado),
    ('estimador', busqueda_random4.best_estimator_)
])

In [66]:
pipeline_estimador_op_best4.fit(X_train, y_train)

Pipeline(steps=[('procesador',
                 FeatureUnion(transformer_list=[('variables_numericas',
                                                 ColumnTransformer(transformers=[('transformador_numerico',
                                                                                  Pipeline(steps=[('imputador_numerico',
                                                                                                   SimpleImputer()),
                                                                                                  ('escalador_numerico',
                                                                                                   MinMaxScaler())]),
                                                                                  Index(['residentes_hogar', 'd2_04_num_hijos'], dtype='object'))])),
                                                ('variables_ordinales',
                                                 Pipeline(steps=[('transfo...
                                                                                                    'g_01_a_num_familiares_consumen_sp_imp_tipo',
                                                                                                    'per_sexo_tipo',
                                                                                                    'vivienda_tipo',
                                                                                                    'd2_03_estado_civil_tipo',
                                                                                                    'g_03_curiosidad_probar_sp_tipo',
                                                                                                    'd_12_c_presenta_problema_expendio_sp_barrio_tipo',
                                                                                                    'g_06_c_posibilidad_conseguir_basuco_tipo',
                                                                                                    'vive_padre_hogar_tipo',
                                                                                                    'vive_madre_hogar_tipo', ...])]))]))])),
                ('estimador',
                 LogisticRegression(C=2.3118996566114403, max_iter=13000,
                                    penalty='l1', solver='saga'))])

In [67]:
pipeline_estimador_op_best4.predict(X_test)

array([0, 2, 0, ..., 2, 0, 4], dtype=int64)

In [68]:
estimador_best4 = pipeline_estimador_op_best4.named_steps['estimador']
resultados["rl_best_4"] = evaluar_modelo(estimador_best4, consumidores_procesado, consumidores.CatRiesgo, 50)
ver_resultados()

,fit_time,score_time,test_score,train_score
rl_exactitud,14.741924,0.002741,0.980400,0.992369
rl_precision,14.511580,0.005860,0.981103,0.992360
rl_exhaustividad,14.600140,0.004940,0.980400,0.992369
rl_f1_score,14.440344,0.005501,0.980343,0.992360
rl_best_exactitud,189.739768,0.001530,0.986000,0.997016
rl_best_precision,183.521629,0.004372,0.986594,0.997018
rl_best_exhaustividad,183.239200,0.004620,0.986000,0.997016
rl_best_f1_score,182.556387,0.004101,0.985960,0.997011
rl_best_2_exactitud,217.689699,0.001902,0.985300,0.996088
rl_best_2_precision,218.118712,0.003961,0.985912,0.996088


### Exportar con joblib el mejor pipeline obtenido

Tenemos que exportar 3 cosas para poder recrear el Pipeline:

**1. El pipeline entrenado.**

Podemos usar la libreria de scikit-learn `joblib` para guardar nuestro pipeline al disco duro. Joblib es una libreria basada en pickle que sirve para guardar objetos de python al disco duro, pero que está optimizado para scikit-learn.

In [26]:
import joblib

In [70]:
joblib.dump(pipeline_estimador_op_best4, 'pipeline_rl_best.pkl')

['pipeline_rl_best.pkl']

In [27]:
clf = joblib.load('C:/Users/Danny/Desktop/Analisis_Modelo/Alisis Danny/4 - Pipelines/Modelo RL/pipeline_rl_best.pkl')

In [72]:
array_predicciones = clf.predict(X_test)

In [73]:
array_predicciones

array([0, 2, 0, ..., 2, 0, 4], dtype=int64)

**2. Los nombres de las columnas**

Necesitamos saber las columnas que espera el pipeline para poder saber cómo tenemos que formatear los datos en el futuro de forma que los reconozca.

In [28]:
import json
with open('columnas_consumidores.json', 'w') as fname:
    consumidores_columnas = consumidores.columns.tolist()
    json.dump(consumidores_columnas, fname)

**3. Los tipos de datos de cada variable**

Necesitamos que las variables de las nuevas observaciones estén en el mismo formato que espera el pipeline.

In [29]:
consumidores_dtypes = consumidores.dtypes
consumidores_dtypes = {col: consumidores[col].dtype for col in consumidores.columns}
joblib.dump(consumidores_dtypes, 'dtypes_consumidores.pkl')

['dtypes_consumidores.pkl']

## DATA PARA GRAFICOS

In [76]:
from collections import Counter
conteo = Counter(array_predicciones)

In [77]:
conteo

Counter({0: 866, 1: 480, 2: 479, 3: 99, 4: 76})

In [78]:
y_test.value_counts()

0    864
1    477
2    475
3    114
4     70
Name: CatRiesgo, dtype: int64

### Importancia de las caracterisiticas

In [30]:
model = clf.named_steps['estimador']

In [31]:
coef = model.coef_

In [32]:
procesador_pipeline = clf.named_steps['procesador']

In [34]:
len(coef[0])

145

In [35]:
nombres_caracteristicas_procesadas = []

for nombre, transformador in procesador_pipeline.transformer_list:
    if hasattr(transformador, 'get_feature_names_out'):
        nombres_generados = transformador.get_feature_names_out()
        nombres_caracteristicas_procesadas.extend(nombres_generados)
    else:
        nombres_caracteristicas_procesadas.append(nombre)

In [37]:
len(nombres_caracteristicas_procesadas)

145

In [38]:
resultados = []

# Iterar sobre cada elemento del arreglo original
for elemento in nombres_caracteristicas_procesadas:
    # Dividir la cadena por el caracter '_' y seleccionar la última parte
    partes = elemento.split('__')
    ultimo_parte = partes[-1]
                
    # Agregar la última parte al nuevo arreglo
    resultados.append(ultimo_parte)

In [45]:
def obtenerDictProcesado(pipeline_columnas,importancia_caracteristicas_dict):
    nuevo_dict_caracteristicas_procesadas = {}
    sumatoria = 0
    promedio = 0
    num_caracteristicas = 0;
    for columna in pipeline_columnas:
        
        if columna == "d2_04_num_hijos":
            nuevo_dict_caracteristicas_procesadas["d2_04_num_hijos"] = importancia_caracteristicas_dict["d2_04_num_hijos"]
        else:
            for clave in importancia_caracteristicas_dict:
                if columna in clave:
                    sumatoria = sumatoria + importancia_caracteristicas_dict[clave]
                    num_caracteristicas = num_caracteristicas + 1
            
            if num_caracteristicas > 1:
                promedio = sumatoria / num_caracteristicas
                nuevo_dict_caracteristicas_procesadas[columna] = promedio
            else:
                nuevo_dict_caracteristicas_procesadas[columna] = sumatoria
            
            sumatoria = 0
            promedio = 0
            num_caracteristicas = 0;
    
    return nuevo_dict_caracteristicas_procesadas

#### Para la un valor 0 en la columna CatRiesgo se obtiene que la importancia es

In [46]:
importancia_caracteristicas_dict_0 = dict(zip(resultados, coef[0]))

In [48]:
dict_procesado_0 = obtenerDictProcesado(consumidores_columnas, importancia_caracteristicas_dict_0)

In [49]:
dict_procesado_0

{'departamento': -0.010320525921174637,
 'residentes_hogar': 0.0,
 'd2_04_num_hijos': 0.0,
 'per_edad_tipo': 3.00015048806396,
 'estrato_tipo': -2.5632680662731486,
 'situacion_tipo': 0.13276957319777521,
 'frecuencia_consumo_marihuana_tipo': -23.357922030525497,
 'frecuencia_consumo_cocaina_tipo': -6.099080425198173,
 'frecuencia_consumo_basuco_tipo': -0.1642048495623858,
 'per_sexo_tipo': 0.7393135178069365,
 'vive_padre_hogar_tipo': 0.4958327886182752,
 'vive_madre_hogar_tipo': 0.5733036206418568,
 'vivienda_tipo': 0.31726514944400497,
 'd_01_aporta_dinero_hogar_tipo': 0.13039517420675792,
 'd_08_estado_salud_tipo': -2.0290067575605057,
 'd_09_deprimido_tipo': 0.04515006646928748,
 'd_10_poco_interes_tipo': 0.35906256913687956,
 'd_11_h_conocimiento_riesgo_fumar_marihuana_frecuentemente_tipo': 0.22551566034728476,
 'd_11_k_conocimiento_riesgo_cocaina_frecuentemente_tipo': 0.18519469269220273,
 'd_11_n_conocimiento_riesgo_fumar_basuco_frecuentemente_tipo': 0.0,
 'd_12_b_presenta_prob

#### Para la un valor 1 en la columna CatRiesgo se obtiene que la importancia es

In [50]:
importancia_caracteristicas_dict_1 = dict(zip(resultados, coef[1]))
dict_procesado_1 = obtenerDictProcesado(consumidores_columnas, importancia_caracteristicas_dict_1)
dict_procesado_1

{'departamento': -0.019100150327224986,
 'residentes_hogar': 0.43174460971551837,
 'd2_04_num_hijos': 0.0,
 'per_edad_tipo': 1.3262133110716816,
 'estrato_tipo': -1.0193791008902457,
 'situacion_tipo': 0.11677531335253158,
 'frecuencia_consumo_marihuana_tipo': -9.338235246304569,
 'frecuencia_consumo_cocaina_tipo': -2.2967895835801246,
 'frecuencia_consumo_basuco_tipo': -0.5925562651641536,
 'per_sexo_tipo': 1.425795433792297,
 'vive_padre_hogar_tipo': 0.27893589770216537,
 'vive_madre_hogar_tipo': 0.35602866058640875,
 'vivienda_tipo': 0.2552840134639567,
 'd_01_aporta_dinero_hogar_tipo': 0.08731603825282595,
 'd_08_estado_salud_tipo': -0.881508084633502,
 'd_09_deprimido_tipo': 0.22222889431911497,
 'd_10_poco_interes_tipo': 0.2261288120539352,
 'd_11_h_conocimiento_riesgo_fumar_marihuana_frecuentemente_tipo': 0.3485589699717841,
 'd_11_k_conocimiento_riesgo_cocaina_frecuentemente_tipo': 0.043438728209825514,
 'd_11_n_conocimiento_riesgo_fumar_basuco_frecuentemente_tipo': 0.131518949

#### Para la un valor 2 en la columna CatRiesgo se obtiene que la importancia es

In [51]:
importancia_caracteristicas_dict_2 = dict(zip(resultados, coef[2]))
dict_procesado_2 = obtenerDictProcesado(consumidores_columnas, importancia_caracteristicas_dict_2)
dict_procesado_2

{'departamento': -0.036795012671000514,
 'residentes_hogar': 0.0,
 'd2_04_num_hijos': 0.0,
 'per_edad_tipo': 0.0,
 'estrato_tipo': 0.0,
 'situacion_tipo': 0.07801021312947116,
 'frecuencia_consumo_marihuana_tipo': 2.274908130618949,
 'frecuencia_consumo_cocaina_tipo': 0.0,
 'frecuencia_consumo_basuco_tipo': 0.15201350445775172,
 'per_sexo_tipo': 0.4463348903321699,
 'vive_padre_hogar_tipo': 0.02740916632716227,
 'vive_madre_hogar_tipo': 0.024588621437246588,
 'vivienda_tipo': 0.05272821295537431,
 'd_01_aporta_dinero_hogar_tipo': 0.2995742266184419,
 'd_08_estado_salud_tipo': 0.0,
 'd_09_deprimido_tipo': 0.3282252691753205,
 'd_10_poco_interes_tipo': 0.029280101370169887,
 'd_11_h_conocimiento_riesgo_fumar_marihuana_frecuentemente_tipo': -0.07541088972319179,
 'd_11_k_conocimiento_riesgo_cocaina_frecuentemente_tipo': 0.0,
 'd_11_n_conocimiento_riesgo_fumar_basuco_frecuentemente_tipo': 0.033666157490434875,
 'd_12_b_presenta_problema_consumo_sp_barrio_tipo': 0.23324187977733843,
 'd_12_

#### Para la un valor 3 en la columna CatRiesgo se obtiene que la importancia es

In [52]:
importancia_caracteristicas_dict_3 = dict(zip(resultados, coef[3]))
dict_procesado_3 = obtenerDictProcesado(consumidores_columnas, importancia_caracteristicas_dict_3)
dict_procesado_3

{'departamento': 0.021163250984475978,
 'residentes_hogar': 0.0,
 'd2_04_num_hijos': 0.0,
 'per_edad_tipo': 0.0,
 'estrato_tipo': 0.010910267907604257,
 'situacion_tipo': -0.03125618989586986,
 'frecuencia_consumo_marihuana_tipo': 8.242715738964986,
 'frecuencia_consumo_cocaina_tipo': 1.2136907252263751,
 'frecuencia_consumo_basuco_tipo': 0.0,
 'per_sexo_tipo': -0.696106111014711,
 'vive_padre_hogar_tipo': 0.0,
 'vive_madre_hogar_tipo': 0.13543875532475688,
 'vivienda_tipo': 0.20451302709412683,
 'd_01_aporta_dinero_hogar_tipo': 0.024416297909048514,
 'd_08_estado_salud_tipo': 0.0816671965550192,
 'd_09_deprimido_tipo': 0.0,
 'd_10_poco_interes_tipo': -0.3654624080143833,
 'd_11_h_conocimiento_riesgo_fumar_marihuana_frecuentemente_tipo': -0.16101740381164129,
 'd_11_k_conocimiento_riesgo_cocaina_frecuentemente_tipo': 0.0,
 'd_11_n_conocimiento_riesgo_fumar_basuco_frecuentemente_tipo': 0.0,
 'd_12_b_presenta_problema_consumo_sp_barrio_tipo': -0.4174257862464335,
 'd_12_c_presenta_proble

#### Para la un valor 4 en la columna CatRiesgo se obtiene que la importancia es

In [53]:
importancia_caracteristicas_dict_4 = dict(zip(resultados, coef[4]))
dict_procesado_4 = obtenerDictProcesado(consumidores_columnas, importancia_caracteristicas_dict_4)
dict_procesado_4

{'departamento': 0.05499255915482961,
 'residentes_hogar': 0.0,
 'd2_04_num_hijos': 0.0,
 'per_edad_tipo': -1.3050127949321242,
 'estrato_tipo': 0.6083504912645291,
 'situacion_tipo': -0.36023915677168755,
 'frecuencia_consumo_marihuana_tipo': 14.533791808678247,
 'frecuencia_consumo_cocaina_tipo': 2.3802099844220517,
 'frecuencia_consumo_basuco_tipo': 0.0,
 'per_sexo_tipo': -1.811187916292461,
 'vive_padre_hogar_tipo': -0.04794645601661087,
 'vive_madre_hogar_tipo': -0.07519923966007064,
 'vivienda_tipo': -0.3066425076975065,
 'd_01_aporta_dinero_hogar_tipo': -0.0317872155276461,
 'd_08_estado_salud_tipo': 0.10748044712413501,
 'd_09_deprimido_tipo': -0.3526138159281847,
 'd_10_poco_interes_tipo': -0.03178756884444377,
 'd_11_h_conocimiento_riesgo_fumar_marihuana_frecuentemente_tipo': -0.46652873411507184,
 'd_11_k_conocimiento_riesgo_cocaina_frecuentemente_tipo': 0.0,
 'd_11_n_conocimiento_riesgo_fumar_basuco_frecuentemente_tipo': -0.19310019645269197,
 'd_12_b_presenta_problema_cons

In [98]:
from sklearn.metrics import precision_score

In [99]:
precision_per_class = precision_score(y_test, array_predicciones, average=None)

In [100]:
precision_per_class

array([0.99538106, 0.98125   , 0.97703549, 0.98989899, 0.90789474])